In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import copy
import pickle

import sys
sys.path.append('./')
sys.path.append('./../../designer/script')

from optimizer.adam import AdamFreeFormOptimizer
from 